# Baseline

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa

import transformers
from transformers import AutoTokenizer
from transformers import TFAutoModel
transformers.logging.set_verbosity_error()

import re
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import argparse
# import wandb
# from wandb.keras import WandbCallback
# wandb.init(project="DACON_235978", name="gnoeyheat")

parser = argparse.ArgumentParser(description="gnoeyheat")
parser.add_argument('--text_pretrained_model', default="roberta", type=str)
parser.add_argument('--text_len', default=300, type=int)
parser.add_argument('--optimizer', default="sgd", type=str) # sgd or adam
parser.add_argument('--learning_rate', default=0.002, type=float)
parser.add_argument('--loss', default='cc', type=str) # cc or fl
parser.add_argument('--label_smoothing', default=0.1, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=30, type=int)
parser.add_argument('--validation_split', default=0.1, type=float)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

# wandb.config.update(args)

os.environ["CUDA_VISIBLE_DEVICES"]="0"

text_pretrained_model = args.text_pretrained_model
text_len = args.text_len
BATCH_SIZE=args.batch_size
EPOCHS=args.epochs
VALIDATION_SPLIT=args.validation_split
SEED=args.seed

def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()

## Text Pretrained Model

In [ ]:
if args.text_pretrained_model == "roberta":
    text_pretrained_model = "klue/roberta-large"
    
tokenizer = AutoTokenizer.from_pretrained(text_pretrained_model)
# tokenizer.truncation_side = 'left'

## Preprocessing

In [ ]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

X_txt = train["overview"]
X_test_txt = test["overview"]

y = train["cat3"]
y_encoder = {key : value for key, value in zip(np.unique(y), range(len(np.unique(y))))}
y = np.array([y_encoder[k] for k in y])

X_txt.shape, X_test_txt.shape, y.shape

In [ ]:
def text_cleaning(df):
    df = df.apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', ' ', x))
    df = df.apply(lambda x : ' '.join(x.split()))
    return df

X_txt = text_cleaning(X_txt)
X_test_txt = text_cleaning(X_test_txt)

X_txt.iloc[0]

In [ ]:
from hanspell import spell_checker

def text_correcting(df):
    correct = []
    for text in tqdm(df):
        if len(text)<500:
            correct.append(spell_checker.check(text).checked)
        else:
            temp = []
            for i in range(0, len(text), 500):
                temp.append(spell_checker.check(text[i:i+500]).checked)
            correct.append(''.join(temp))
    return pd.Series(correct, name="overview")
    
X_txt = text_correcting(X_txt)
X_test_txt = text_correcting(X_test_txt)

train["overview"] = X_txt
test["overview"] = X_test_txt
train.to_csv("data/pp_train.csv", index=False)
test.to_csv("data/pp_test.csv", index=False)

X_txt.iloc[0]

In [ ]:
train["len"] = train["overview"].apply(tokenizer.tokenize).apply(len)
test["len"] = test["overview"].apply(tokenizer.tokenize).apply(len)

train["len"].median(), test["len"].median()

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(
        self,
        sentence,
        labels,
        batch_size=BATCH_SIZE,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence = sentence
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        self.tokenizer = tokenizer
        self.indexes = np.arange(len(self.sentence))
        self.on_epoch_end()

    def __len__(self):
        return len(self.sentence) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence = self.sentence[indexes]

        encoded = self.tokenizer.batch_encode_plus(
            sentence.tolist(),
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            max_length=text_len,
            return_tensors="tf",
            return_token_type_ids=True,
            return_attention_mask=True,
        )

        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(SEED).shuffle(self.indexes)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_txt, y, test_size=VALIDATION_SPLIT, random_state=SEED, stratify=y)

y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
train_ds = DataGenerator(
    X_train.values, y_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
)
val_ds = DataGenerator(
    X_val.values, y_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

## Modelling

In [ ]:
input_ids = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name="input_ids"
)
attention_masks = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name="attention_masks"
)
token_type_ids = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name="token_type_ids"
)

bert_model = TFAutoModel.from_pretrained(text_pretrained_model, from_pt=True)
bert_model.trainable = True

bert_output = bert_model(
    input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
)

x = bert_output.last_hidden_state
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)

output = layers.Dense(y_train.shape[1], activation="softmax")(x)

model = tf.keras.models.Model(
    inputs=[input_ids, attention_masks, token_type_ids], outputs=output
)

if args.optimizer == "sgd":
    optim = tf.keras.optimizers.SGD(
        learning_rate=lr, momentum=0.9
    )
if args.loss == "cc":
    loss_function = tf.keras.losses.CategoricalCrossentropy(
        label_smoothing=args.label_smoothing
    )

model.compile(
    optimizer=optim,
    loss=loss_function,
    metrics=tfa.metrics.F1Score(num_classes=y_train.shape[1], average="weighted")
)

model.summary()

## Training

In [ ]:
checkpoint_path = f"load_model/{parser.description}"

def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback = [
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor="val_f1_score",
        save_best_only=True,
        save_weights_only=True,
        mode="max",
    )
]

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[callback],
    validation_data=val_ds,
)

In [ ]:
acc = history.history['f1_score']
val_acc = history.history['val_f1_score']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(acc, label='Training Weighted-F1')
plt.plot(val_acc, label='Validation Weighted-F1')
plt.legend(loc='lower right')
plt.title('Training and Validation Weighted-F1')
plt.show()

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
val_weighted_f1 = model.evaluate(val_ds)[1]
print(f"val_weighted_f1: {val_weighted_f1}")

## Inference

In [ ]:
test_ds = DataGenerator(
    X_test_txt.values, None,
    batch_size=16,
    shuffle=False,
    include_targets=False,
)

In [ ]:
pred_prob = []
for i in range(test_ds.__len__()):
    pred_prob.append(model.predict(test_ds.__getitem__(i)))
pred_prob = np.vstack(pred_prob)
pred = np.argmax(pred_prob, axis=1)

y_decoder = {value : key for key, value in y_encoder.items()}
result = np.array([y_decoder[v] for v in pred])

pd.Series(result).value_counts()

In [ ]:
submission = pd.read_csv("data/sample_submission.csv")
submission["cat3"] = result
submission.to_csv(f"{parser.description}.csv", index=False)